In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os, sys
from tensorflow.keras.layers import Input

#find root directory and add to path
root_index = os.getcwd().rfind('MotorNet') + len('MotorNet')+1
root_directory = os.path.dirname(os.getcwd()[:root_index])
sys.path.append(root_directory)

from MotorNet.plants import PlantWrapper
from MotorNet.plants.skeletons import TwoDofArm
from MotorNet.plants.muscles import ReluMuscle
from MotorNet.utils.plotor import plot_pos_over_time
from MotorNet.nets.losses import empty_loss, position_loss, activation_squared_loss
from MotorNet.nets.callbacks import BatchLogger
from MotorNet.nets.layers import GRUController
from MotorNet.tasks import TaskStaticTarget
from MotorNet.nets.models import MotorNetModel

%load_ext autoreload
%autoreload 2

#tf.debugging.enable_check_numerics()
print('tensorflow version: ' + tf.__version__)
print('numpy version: ' + np.__version__)


tensorflow version: 2.6.0
numpy version: 1.19.5


In [2]:
plant = PlantWrapper(skeleton=TwoDofArm(),muscle_type=ReluMuscle(), proprioceptive_delay=0.04, visual_delay=0.13)

plant.add_muscle(path_fixation_body=[0, 1], path_coordinates=[[5,  5],[0, 0]], name='ur', max_isometric_force=500)
plant.add_muscle(path_fixation_body=[0, 1], path_coordinates=[[-5, 5],[0, 0]], name='ul', max_isometric_force=500)
plant.add_muscle(path_fixation_body=[0, 1], path_coordinates=[[5, -5],[0, 0]], name='dr', max_isometric_force=500)
plant.add_muscle(path_fixation_body=[0, 1], path_coordinates=[[-5,-5],[0, 0]], name='dl', max_isometric_force=500)

In [7]:
cell = GRUController(plant=plant, n_units=50, kernel_regularizer=10**-6, name='cell')
task = TaskStaticTarget(cell, delay_range=[0.3, 0.6])

# declare inputs
inputs = {key:Input((None, val,), name=key) for key, val in task.get_input_dim().items()}
state0 = [Input((plant.output_dim, ), name='joint0'),
          Input((plant.output_dim, ), name='cartesian0'),
          Input((plant.muscle_state_dim, plant.n_muscles, ), name='muscle0'),
          Input((plant.geometry_state_dim, plant.n_muscles, ), name='geometry0'),
          Input((plant.n_muscles * 2, plant.proprioceptive_delay, ), name='proprio_feedback0'),
          Input((plant.space_dim, plant.visual_delay, ), name='visual_feedback0')]
state0.extend([Input((n, ), name='gru' + str(k) + '_hidden0') for k, n in enumerate(cell.n_units)])

# wrap cell in an RNN layer
rnn = tf.keras.layers.RNN(cell=cell, return_sequences=True, name='RNN')
states_out = rnn(inputs, initial_state=state0)
control_rnn = MotorNetModel(inputs=[inputs, state0], outputs=states_out, name='controller', task=task)

# and compile
control_rnn.compile(optimizer=tf.optimizers.Adam(clipnorm=1.), loss=task.losses, loss_weights=task.loss_weights)
batchlog_callback = BatchLogger()
control_rnn.summary()

AttributeError: in user code:

    C:\Users\decomite\Documents\SuperlabMotornet\MotorNet\MotorNet\nets\layers.py:140 call  *
        jstate, cstate, mstate, gstate = self.plant(u, jstate, mstate, gstate, **inputs)
    C:\Users\decomite\Documents\SuperlabMotornet\MotorNet\MotorNet\plants\__init__.py:131 __call__  *
        return self._call_fn((muscle_input, joint_state, muscle_state, geometry_state, endpoint_load, joint_load))
    C:\Users\decomite\Anaconda3\lib\site-packages\keras\engine\base_layer.py:1037 __call__  **
        outputs = call_fn(inputs, *args, **kwargs)
    C:\Users\decomite\Anaconda3\lib\site-packages\keras\layers\core.py:903 call
        result = self.function(inputs, **kwargs)
    C:\Users\decomite\Documents\SuperlabMotornet\MotorNet\MotorNet\plants\__init__.py:76 <lambda>
        self._call_fn = Lambda(lambda x: self.call(*x))
    C:\Users\decomite\Documents\SuperlabMotornet\MotorNet\MotorNet\plants\__init__.py:175 call
        self.integrate(muscle_input, joint_st

In [ ]:
dataset_size = 2000 *32
n_t = int(1 / plant.dt)

with tf.device('/cpu:0'):
    [inputs, targets, init_states] = task.generate(batch_size=dataset_size, n_timesteps=n_t)
    control_rnn.fit([inputs, init_states], targets, epoch=1, batch_size=32, verbose=1, callbacks=[batchlog_callback], shuffle=False)
    

In [ ]:
if 'loss'in batchlog_callback.history:
    last_loss = batchlog_callback.history['loss'][-1]
    print('last loss value = ' + str(last_loss))
    plt.plot(batchlog_callback.history['loss'])
    plt.xlabel('batch')
    plt.ylabel('loss')
    #plt.semilogy()
    plt.show()

In [ ]:
def grad(model, model_inputs, y_true, loss):
    with tf.GradientTape() as tape:
        model_pred = model(model_inputs, trainin=False)
        y_pred = model.pred['cartesian position']
        loss_value = loss(y_true=y_true, y_pred=y_pred)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)
[inputs_sample, targets_sample, init_states_sample]=task.generate(batch_size=1, n_timesteps=n_t)
loss_val, gradients = grad(control_rnn, [inputs_sample, init_states_sample], targets_sample, position_loss())
#gradients

In [ ]:
[inputs_sample, targets_sample, init_states_sample]=task.generate(batch_size=20,n_timesteps=n_t)
results=control_rnn([inputs_sample, init_states_sample],training=False)

j_results = results['joint position']
c_results = results['cartesian position']
m_results = results['muscle state']

plt.figure(figsize=16,4)).set_tight_layout(True)
plt.subplot(131)
plot_pos_over_time(cart_results=c_results)
plt.scatter(targets_sample[:,0,0], targets_sample[:,0,1])

plt.subplot(132)
plot_pos_over_time(cart_results=c_results - np.expand_dims(targets_sample[:,-1,:],axis=1))
plt.xlabel('cartesian distance to target (x)')
plt.ylabel('cartesian distance to target (y)')

plt.subplot(133)
plt.scatter(j_results[:,-1,0].numpy().T, j_results[:,-1,1].numpy().T)
plt.xlabel('shoulder end-reach position (rad)')
plt.ylabel('elbow end-reach position (rad)')
plt.show()

In [ ]:
#The goal here is to represent only one trial 

trial = 0
plt.figure(figsize=(14, 2.5)).set_tight_layout(True)

plt.subplot(141)
plt.plot(j_results[trial,:,0].numpy().squeeze(),label='sho')
plt.plot(j_results[trial,:,1].numpy().squeeze(),label='elb')
plt.legend()
plt.xlabel('time (ms)')
plt.ylabel('angle (rad)')

plt.subplot(142)
plt.plot(j_results[trial,:,2].numpy().squeeze(),label='sho')
plt.plot(j_results[trial,:,3].numpy().squeeze(),label='elb')
plt.legend()
plt.xlabel('time(ms)')
plt.ylabel('angle velocity (rad/sec)')

plt.subplot(143)
plt.plot(m_results[trial,:,0,:].numpy().squeeze())
plt.xlabel('time (ms)')
plt.ylabel('activation (a.u.)')

plt.subplot(144)
plt.plot(m_results[trial,:,1,:].numpy().squeeze())
plt.xlabel('time (ms)')
plt.ylabel('force (N)')
plt.show()